<a href="https://colab.research.google.com/github/Bhar8at/Stock-Analysis/blob/main/quantrecruitment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Companies :
- BlackBerry Ltd ( Primary Company )
- Bank of Baroda
- Bath and Body Works Inc
- Boeing Company
- Barnes Group

In [ ]:
%pip install yfinance
%pip install pandas
%pip install plotly

In [ ]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px


In [ ]:
BB = yf.Ticker("BB")
hist = BB.history(period = "5y")

In [ ]:
# Displaying Daily , Weekly and Monthly Price Charts
def Price_Charts(df):
    # Daily View
    fig1 = go.Figure(
    data = [
        go.Candlestick(
        x = df.index,
        open = hist["Open"],
        high = hist["High"],
        low = hist["High"],
        close = hist["Close"]
            )
        ]
    )
    fig1.show()

    # Weekly View
    df_w = df.resample('W').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last'
})
    fig2 = go.Figure(
    data = [
        go.Candlestick(
        x = df_w.index,
        open = hist["Open"],
        high = hist["High"],
        low = hist["High"],
        close = hist["Close"]
        )
    ]
)
    fig2.show()
    # Monthly View
    df_m = df.resample('M').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last'
})
    fig3 = go.Figure(
    data = [
        go.Candlestick(
        x = df_m.index,
        open = hist["Open"],
        high = hist["High"],
        low = hist["High"],
        close = hist["Close"]
        )
    ]
)
    fig3.show()

Price_Charts(hist)

# Inferences:
- a Sudden spike in prices in the month of January 2021
  - On further investigation realised that it was caused as a result of short squeeze coordinated by a bunch of investors on reddit and not as correlated with the strategic advances by the company




In [ ]:
# Displaying Lifetime High and Low
def Lifetime_High_Low(df):
    print(
    f"Lifetime High : {df['High'].max()} \nLifetime Low : {df['Low'].min()}"
)

# Displaying High and Low in the past 52 weeks
def High_Low(df):
    df_w = df.resample('W').agg({
    'High': 'max',
    'Low': 'min',
})
    print(
    f"High in past 52 weeks : {df_w.tail(52)['High'].max()} \nLow in past 52 weeks : {df_w.tail(52)['Low'].min()}"
)


Lifetime_High_Low(hist)
High_Low(hist)

Lifetime High : 28.770000457763672 
Lifetime Low : 2.009999990463257
High in past 52 weeks : 5.650000095367432 
Low in past 52 weeks : 2.009999990463257


In [ ]:
SnP = yf.Ticker("^GSPC") # Using the S&P 500 index for Risk measure calculations / What is S&P though?
benchmark = SnP.history(period = "5y")


# Calculating the Standard Deviation and Beta

def Std_of_Returns(df):
    df["Daily Returns"] = df["Close"].pct_change()
    print(f"Standard deviation of the Returns : {df['Daily Returns'].std()}")

def beta(df, bk):
    df["Daily Returns"] = df["Close"].pct_change()
    bk["Daily Returns"] = bk["Close"].pct_change()
    cov = df["Daily Returns"].cov(bk["Daily Returns"])
    beta = cov/(bk["Daily Returns"].var())
    print(f"Beta Value : {beta}")


Std_of_Returns(hist)
beta(hist, benchmark)


Standard deviation of the Returns : 0.04449751359858442
Beta Value : 1.32073362204372


In [ ]:
# Calculating Return measures (Daily , Monthly , Annual)

def ReturnMeasures(df):
    # Daily Returns
    df["Daily Return (Return measure)"] = ((df["Close"] - df["Open"])/df["Open"])*100
    fig1 = px.line(
        df,
        x = df.index,
        y = df["Daily Return (Return measure)"],
        title = "Daily Returns"
    )
    fig1.show()

    # Monthly Returns
    df_m = df.resample('M').agg({
    'Open': 'first',
    'Close': 'last'
    })

    df_m["Monthly Returns"] = ((df_m["Close"] - df_m["Open"])/df_m["Open"])*100
    fig1 = px.line(
        df_m,
        x = df_m.index,
        y = df_m["Monthly Returns"],
        title = "Monthly Returns"
    )
    fig1.show()

    # Annual Returns calculated by using CAGR
    CAGR = (df["Close"].iloc[-1]/df["Close"].iloc[0])**(1/5) - 1
    print(f"CAGR of the stock during the 5Y time period is {CAGR} ")

ReturnMeasures(hist)


/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



CAGR of the stock during the 5Y time period is -0.20720538797395005 


- Clearly from the CAGR we can infer that the stock has performed poorly over the 5 years.
- We can See from the monthly Returns that the most returns was made in the start of 2021
  - So if a person were to invest during the Lowest point of BB (i.e during March 2020) by the start of 2021 they would have quadrupled their investment

In [ ]:
# Moving Averages
def SMA(df, n):
    df = df.tail(n)
    print(f"The SMA for {n} days is : {df['Close'].sum()/n}")

SMA(hist, 20)
SMA(hist, 50)

The SMA for 20 days is : 2.3014999866485595
The SMA for 50 days is : 2.3632000207901003


- Here the 20 day moving average is our short term trend and the 50 day moving average is our Long term trend
- Clearly our short term trend has dipped below the long term trend indicating a bearish market.
- If the short term trend were to rise back then it could indicate a potential trend reversal too

In [ ]:
# Calculating the Relative Strength Index

def RSI(df):
    df["Daily Return"] = df["Close"].pct_change()
    sg = sl = 0 # Sum of Gains and losses
    for i in range(1,15):
        if df["Daily Return"].iloc[-i] < 0:
            sl += df["Daily Return"].iloc[-i]
        else:
            sg += df["Daily Return"].iloc[-i]
    Ag  = sg/14
    Al = -sl/14
    RS = Ag/Al
    RSI = 100 - (100/(1+RS))
    print(f"The RSI is {RSI}")

RSI(hist)

The RSI is 55.22469848470588


- a RSI of > 70 indicates an overbought stock
- So our stock is clearing approaching that region

In [ ]:
# Calculating the Moving Averages Convergence Divergence

def EMA(df,n):
    return df['Close'].ewm(span=n, adjust=False).mean()

def MACD(df):
    EMA_12 = EMA(df,12)
    EMA_26 = EMA(df,26)
    print(f"EMA_12 is given by {EMA_12}")
    print(f"EMA_26 is given by {EMA_26}")
    MACD = EMA_12 - EMA_26
    signal_line = MACD.ewm(span=9, adjust=False).mean()
    MACDH = MACD - signal_line
    result = pd.DataFrame({
        'MACD Line': MACD,
        'Signal Line': signal_line,
        'MACD Histogram': MACDH
    })

    fig = px.line(result, x=result.index, y=['MACD Line', 'Signal Line'], title="MACD Line and Signal Line")
    fig.show()

    fig_hist = px.bar(result, x=result.index, y='MACD Histogram', title="MACD Histogram")
    fig_hist.show()

MACD(hist)


EMA_12 is given by Date
2019-09-05 00:00:00-04:00    7.280000
2019-09-06 00:00:00-04:00    7.266154
2019-09-09 00:00:00-04:00    7.222130
2019-09-10 00:00:00-04:00    7.211033
2019-09-11 00:00:00-04:00    7.257028
                               ...   
2024-08-28 00:00:00-04:00    2.338781
2024-08-29 00:00:00-04:00    2.340507
2024-08-30 00:00:00-04:00    2.341968
2024-09-03 00:00:00-04:00    2.329357
2024-09-04 00:00:00-04:00    2.321764
Name: Close, Length: 1258, dtype: float64
EMA_26 is given by Date
2019-09-05 00:00:00-04:00    7.280000
2019-09-06 00:00:00-04:00    7.273334
2019-09-09 00:00:00-04:00    7.251605
2019-09-10 00:00:00-04:00    7.244079
2019-09-11 00:00:00-04:00    7.263777
                               ...   
2024-08-28 00:00:00-04:00    2.343960
2024-08-29 00:00:00-04:00    2.344408
2024-08-30 00:00:00-04:00    2.344822
2024-09-03 00:00:00-04:00    2.338539
2024-09-04 00:00:00-04:00    2.334202
Name: Close, Length: 1258, dtype: float64


/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



- MACD and Signal Line is used for showing us the momentum of certain trends. i.e To see whether a certain trend is beggining to hit a fullstop or whether a certain trend is going to strengthen further
- But in the Above plots The MACD Line seems to be oscillating above and below the Signal Line which gives us no meaningful conclusions
- Only if the MACD Line is evidently above the signal line / Below the Signal Line can we conclude that the Trend is going to be continuing / Hitting a reversal respectively

# Ratio Analysis



In [ ]:
balance_sheet = BB.balance_sheet
income_statement = BB.financials
cashflow = BB.cashflow

# Calculating the P/E Ratio
EPS = BB.info['trailingEps']
current_price = hist["Close"].iloc[0]
PE_ratio = current_price/EPS
print(f"P/E Ratio : {PE_ratio}")

# Calculating the P/B Ratio
StockHolder_Equity = balance_sheet.loc['Stockholders Equity'].iloc[0]
Shares_Outstanding = balance_sheet.loc['Ordinary Shares Number'].iloc[0]
Book_value_per_share = StockHolder_Equity/Shares_Outstanding
PB_ratio = current_price/Book_value_per_share
print(f"P/B Ratio : {PB_ratio}")

# Calculating the EV/EBITDA Ratio
Total_debt = balance_sheet.loc["Total Debt"].iloc[0]

# Calculating the Enterprise Value
Market_Capitalization = current_price * Shares_Outstanding
cash_and_equiv = balance_sheet.loc['Cash Cash Equivalents And Short Term Investments'].iloc[0] + balance_sheet.loc['Other Short Term Investments'].iloc[0]
preferred_equity = balance_sheet.get('Preferred Stock', 0)
minority_interest = balance_sheet.get('Minority Interest', 0)
Enterprise_Value = Market_Capitalization + Total_debt + preferred_equity + minority_interest - cash_and_equiv

# Calculating the EBITA
net_income = income_statement.loc['Net Income'].iloc[0]
interest_expense = income_statement.get('Interest Expense', 0)
tax_expense = income_statement.get('Income Tax Expense', 0)
depreciation_amortization = income_statement.get('Depreciation & Amortization', 0)
EBITDA = net_income + interest_expense + tax_expense + depreciation_amortization


EBITDA_ratio = Enterprise_Value/EBITDA
print(f"ev - EBITDA Ratio : {EBITDA_ratio}")

P/E Ratio : -26.962963740030922
P/B Ratio : 5.527851813847818
ev - EBITDA Ratio : -32.63548467343005


- The negative yet high Value in the P/E Ratio indicate that the company has been incurring loses but despite this investors seem to be overvaluing the company
- Talking about the P/B ratio, since it's > 1 it is being valued higher than the book value by investors. the market seems to have a positive outlook on the company's future
- But since the negative EV-EBITDA ratio indicates a negative indicates EBITDA which means the company is failing to provide positive cash flow and is running on a loss

In [ ]:
# Calculating Profitability Ratios
# Return on Equity for this year
ROE = income_statement.loc['Net Income'].iloc[0]/balance_sheet.loc['Stockholders Equity'].iloc[0]
print(f"Return on Equity : {ROE}")
# Return on Assets
ROA = income_statement.loc['Net Income'].iloc[0]/balance_sheet.loc['Total Assets'].iloc[0]
print(f"Return on Assets : {ROA}")
# Profit Margin
Profit_Margin = income_statement.loc['Net Income'].iloc[0]/income_statement.loc['Total Revenue'].iloc[0]
print(f"Profit Margin : {Profit_Margin}")

Return on Equity : -0.16752577319587628
Return on Assets : -0.0931899641577061
Profit Margin : -0.15240328253223914


In [ ]:
# Growth Parameters
# Revenue Growth This year compared to last year
RVG = (income_statement.loc['Total Revenue'].iloc[0] - income_statement.loc['Total Revenue'].iloc[1] ) / income_statement.loc['Total Revenue'].iloc[1]
RVG = RVG*100
print(f"Revenue Growth : {RVG}")

# Earnings per share Growth
EPS0  = income_statement.loc['Net Income'].iloc[0]/balance_sheet.loc['Ordinary Shares Number'].iloc[0]
EPS1  = income_statement.loc['Net Income'].iloc[1]/balance_sheet.loc['Ordinary Shares Number'].iloc[1]
EPSG = (EPS0 - EPS1)/EPS1
EPSG = EPSG*100
print(f"Earnings per share Growth : {EPSG}")


Revenue Growth : 30.03048780487805
Earnings per share Growth : -82.50149902757535


In [ ]:
# Cash Flow parameters
# Free Cash Flow
FCF = cashflow.loc['Operating Cash Flow'].iloc[0] - cashflow.loc['Capital Expenditure'].iloc[0]
print(f"Free Cash Flow : {FCF}")

# Cash Conversion Cycle (Fix this part of the code )
## Days Inventory Outstanding
DIO = balance_sheet.loc['Inventory'].iloc[0]/income_statement['Cost of Revenue'].iloc[0]
DIO = DIO * 365
print(f"Days Inventory Outstanding : {DIO}")


Free Cash Flow : 18000000.0


KeyError: 'Inventory'

In [ ]:
# Working Capital Management
## Current Ratio
CR = balance_sheet.loc['Total Assets'].iloc[0]/balance_sheet.loc['Current Liabilities'].iloc[0]
print(f"Current Ratio : {CR}")

## Quick Ratio # Fix this code as well (Find How to Calculate Inventory)
QR = balance_sheet.loc['Total Assets'].iloc[0] - Inventory /balance_sheet.loc['Current Liabilities'].iloc[0]
print(f"Quick Ratio : {QR}")

Current Ratio : 3.918539325842697


NameError: name 'Inventory' is not defined

# Comparing the Stock Price Performance with Firm's Financial Performance

In [ ]:
eps = BB.info['trailingEps']
eps

-0.27

In [ ]:
df_y = hist.resample('Y').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last'
})

# modifying the dataframes to fit with the plot

df_y = df_y.tail(5)
df_y.sort_index(inplace=True, ascending = False)
df_y

SharesPerYear = balance_sheet.loc['Ordinary Shares Number'][:-1]
EPSPerYear = income_statement.loc['Net Income']/balance_sheet.loc['Ordinary Shares Number'][:-1]
EPSPerYear = EPSPerYear.to_frame()
EPSPerYear.columns = ['EPS']

fig1 = px.line(
    df_y,
    x = df_y.index,
    y = df_y["Close"],
    title = "BB Stock Price (Blue) and BB Earnings (Orange)",
)

fig2 = px.line(
    EPSPerYear,
    x = EPSPerYear.index,
    y = 'EPS',
    title = "BB Income Statement",
)

fig2.update_traces(line=dict(color='orange'), name='EPS')
fig1.add_trace(fig2.data[0])

fig1.show()


/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



# Inferences
- With increase in the value of the stock upto the start of 2022 there followed a steady increase in Earnings as well
- The Earnings of the firm seem to be moving in congruence with the price until we reach the year 2024
- Although the stock price seems to be stagnant and slightly decreasing Blackberry seems to be pushing themselves out of their losses.

